Digital filter initialization is one way around the mass imbalance issues we have with the NG-Aqua input data. 

First, we should see if the transient problems in the boundary layer disappear at the end of 3-6 hour integration.

In [ ]:
import holoviews as hv
hv.extension('bokeh')
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 
%opts QuadMesh[width=400,  height=200, colorbar=True](cmap='viridis')
%opts Curve[width=500, height=int(500/1.61)]

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from sam.case import get_ngqaua_ic, InitialConditionCase
import xarray as xr
from os.path import join
import numpy as np
from uwnet.thermo import layer_mass


def process_ic(ic, **kwargs):

    # open initial conditions
    case = InitialConditionCase(ic=ic)

    # configure the sam run
    case.prm['parameters'].update({
        'dosgs': False,
        'dowally': True,
        'nstop': 180,
        'nsave3d': 30,
        'nsave2d': 30,
        'nsave2dstart': 0 ,
        'nstat': 15,
        'nstatfrq': 1,
        'dt': 120,
    })
    
    case.prm['parameters'].update(kwargs)

    case.save()

    # Run the model
    print(case.path)
    !cd {case.path} &&  bash -c './run.sh > out  2> err'
    # !cd {case.path} &&  bash -c './run.sh'

    # open the 3d outputs
    data_3d = xr.open_mfdataset(join(case.path, 'OUT_3D', '*.nc'), concat_dim='time').load()
    assert len(data_3d.time) > 1
    
    data_2d = xr.open_mfdataset(join(case.path, 'OUT_2D', '*.nc'), concat_dim='time').load()
    
    stat = xr.open_mfdataset(join(case.path, 'OUT_STAT', '*.nc'), concat_dim='time').load()
    
    # clean up the working directory
    !rm -rf {case.path}
    
    return data_3d, data_2d, stat


def plot_diffs(sam_processed):
    dsam = sam_processed.diff('time')

    # get the surface level
    dsam_sfc = dsam.isel(z=0)
    
    lay = hv.Dataset(dsam_sfc.TABS).to.image(["x", "y"], label="T(i+1) - T(i)") \
     + hv.Dataset(dsam_sfc.QT).to.image(["x", "y"], label="QT(i+1) - Qt(i)")

    return lay.cols(1)

In [ ]:
NGROOT = "/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/"
STATFILE = join(NGROOT, 'stat.nc')

stat = xr.open_mfdataset(STATFILE)
rho = stat.RHO[0]

# layer mass
dm = layer_mass(rho)

# get the initial condition at first time step
# the winds are staggered
ic = get_ngqaua_ic(NGROOT, 0)

In [ ]:
ic.TABS[0].plot()

In [ ]:
# Process this initial conditions using SAM
data_3d, data_2d, stat = process_ic(ic, dt=120, nstop=10, nsave3d=1, nsave2d=1, nstat=1)

Here is the timestep-to-timestep difference of temperature at the first height level:

In [ ]:
data_3d.TABS.diff('time').isel(z=0).plot(col='time', col_wrap=4)

And for humidity:

In [ ]:
data_3d.QT.diff('time').isel(z=0).plot(col='time', col_wrap=4)

As you can see all the zonal-banding is gone! 